<a href="https://colab.research.google.com/github/shpotes/traffic-counter/blob/dev/notebooks/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
%cd /content/
!rm -rf traffic-counter
!git clone https://github.com/shpotes/traffic-counter -b dev
%cd traffic-counter/notebooks

/content
Cloning into 'traffic-counter'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 972 (delta 34), reused 45 (delta 15), pack-reused 905
Receiving objects: 100% (972/972), 38.22 MiB | 25.04 MiB/s, done.
Resolving deltas: 100% (169/169), done.
/content/traffic-counter/notebooks


In [9]:
!pip install -r ../requirements.txt

In [1]:
import sys
import os
import json
from itertools import groupby
from operator import itemgetter
from typing import List, Tuple, Dict

import gin
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.models import Model, clone_model
from tensorflow.keras.layers import Input, Conv2D, Concatenate, MaxPool2D, Activation, Reshape, ZeroPadding2D
from tensorflow.keras.applications import VGG16

sys.path.append('..')

from vehicle_nowcasting.data.data_loader import build_source_from_metadata, make_dataset, generate_anchors
from vehicle_nowcasting.data.generate_anchors import generate_anchors as gen_anch
from vehicle_nowcasting.utils import iou, plot_bb, change_box_order, compute_stride_from_receptive_field

%matplotlib inline

In [2]:
main_dir = '..'
data_dir = os.path.join(main_dir, 'data')
metadata = pd.read_csv('../data/metadata_test.csv')
train = metadata[metadata.split == 'train']
val = metadata[metadata.split == 'val']
label_map = json.load(open('../data/label_map.json', 'r'))

train_sources = build_source_from_metadata(train, label_map, data_dir, mode='train')
val_sources = build_source_from_metadata(val, label_map, data_dir, mode='val')

In [3]:
train_ds = make_dataset(train_sources, mode='rpn', num_epochs=1)

W0917 18:08:34.027887 140136917493568 deprecation.py:323] From /home/santiago/anaconda3/envs/CV/lib/python3.6/site-packages/tensorflow_core/python/ops/array_ops.py:1486: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


15


In [4]:
def RPN(inputs, k):
    x = ZeroPadding2D(1)(inputs)
    x = Conv2D(256, kernel_size=(3, 3),  
               activation='relu',
               name='window')(x)
    cls = Conv2D(k, kernel_size=(1, 1),
                 activation='sigmoid',
                 name='cls_head')(x)
    reg = Conv2D(4 * k, kernel_size=(1, 1),
                 name='reg_head')(x)
    return Concatenate()([cls, reg])

In [5]:
k = 15
inputs = Input(shape=(224, 224, 3))
backbone_model = VGG16(input_tensor=inputs,
                       weights='imagenet',
                       include_top=False)
x = backbone_model.output
model = Model(inputs=inputs, outputs=RPN(x, k))

In [6]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 224, 224, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 224, 224, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 112, 112, 64) 0           block1_conv2[0][0]               
______________________________________________________________________________________________

In [7]:
y_true = next(iter(train_ds.batch(1)))[1]
p_true = y_true[:, :, :, :2 * k]
t_gt = y_true[:, :, :, 2 * k: 6 * k]
t_bb = y_true[:, :, :, 6 * k:]

In [8]:
print(p_true[:, :, :, 1::2].shape) # 1 if positive example
tf.where(p_true[:, :, :, 1::2])

(1, 7, 7, 15)


<tf.Tensor: id=1127, shape=(11, 4), dtype=int64, numpy=
array([[ 0,  0,  4, 12],
       [ 0,  2,  3,  4],
       [ 0,  3,  2,  1],
       [ 0,  3,  3,  4],
       [ 0,  4,  1, 12],
       [ 0,  4,  2,  0],
       [ 0,  5,  1,  1],
       [ 0,  5,  1,  3],
       [ 0,  5,  1,  6],
       [ 0,  5,  2,  0],
       [ 0,  5,  3,  0]])>

In [9]:
p_mask = p_true[:, :, :, 1::2]

In [10]:
p_true = tf.reduce_max(tf.reshape(p_true, (1, 7, 7, k, 2)), axis=-1)

In [11]:
y_pred = model.predict(tf.zeros((1, 224, 224, 3)))
p_pred = y_pred[:, :, :, :k]
t_pred = y_pred[:, :, :, k:]

In [12]:
tf.reduce_sum(tf.losses.binary_crossentropy(p_true, p_pred))

<tf.Tensor: id=1314, shape=(), dtype=float32, numpy=34.340233>

In [7]:
def reg_parametrization(_t, t_a):
    _t = tf.cast(tf.reshape(_t, (-1, 7, 7, k, 4)), tf.float64)
    t_a = tf.cast(tf.reshape(t_a, (-1, 7, 7, k, 4)), tf.float64)
    t_xy = _t[:, :, :, :, :2]
    t_wh = _t[:, :, :, :, 2:]
    t_a_xy = t_a[:, :, :, :,  :2]
    t_a_wh = t_a[:, :, :, :, 2:]
    
    t_xy = (t_xy - t_a_xy) / t_wh

    t_wh = tf.math.log(t_wh / t_a_wh)

    t = tf.concat([t_xy, t_wh], axis=-1)

    t = tf.where(tf.math.is_nan(t), tf.zeros_like(t), t)
    t = tf.where(tf.math.is_inf(t), tf.zeros_like(t), t)
    
    return t

In [14]:
t_true = reg_parametrization(t_gt, t_bb)
t_pred = reg_parametrization(t_pred, t_bb)

In [8]:
def l1_smooth(x):
    return tf.where(tf.math.abs(x) < 1, 0.5 * x * x, tf.math.abs(x) - 0.5)

In [16]:
tf.reduce_sum(tf.reshape(p_mask, (1, 7, 7, k, 1)) * l1_smooth(t_true - t_pred))

<tf.Tensor: id=1396, shape=(), dtype=float64, numpy=3197.066420150232>

In [13]:
def RPN_loss(k, y_true, y_pred):
    p_true = y_true[:, :, :, :2 * k]
    t_gt = y_true[:, :, :, 2 * k: 6 * k]
    t_bb = y_true[:, :, :, 6 * k:]
    
    p_pred = y_pred[:, :, :, :k]
    t_pred = y_pred[:, :, :, k:]
    
    p_mask = p_true[:, :, :, 1::2]
    p_true = tf.reduce_max(tf.reshape(p_true, (-1, 7, 7, k, 2)), axis=-1)
    
    cls_loss = tf.losses.binary_crossentropy(p_true, p_pred)
    cls_loss = tf.identity(cls_loss, name='cls_loss')
    
    t_true = reg_parametrization(t_gt, t_bb)
    t_pred = reg_parametrization(t_pred, t_bb)
    
    reg_loss = l1_smooth(t_true - t_pred)
    reg_loss = tf.cast(reg_loss, tf.float32)
    
    reg_loss = tf.identity(reg_loss, name='reg_loss')
    
    p_true = tf.reshape(p_true, (-1, 7, 7, k, 1))
    
    return 1 * tf.math.reduce_sum(cls_loss) # +  0 * tf.math.reduce_sum(p_true * reg_loss)

In [ ]:
from tensorflow.keras.callbacks import TensorBoard

adam = tf.optimizers.Adam(learning_rate=3e-4)
model.compile(optimizer=adam,
              loss=lambda y_true, y_pred: RPN_loss(15, y_true, y_pred))

hist = model.fit_generator(train_ds.batch(1).repeat(5))

 5/10 [==============>...............] - ETA: 13s - loss: nan